In [54]:
using Pkg
Pkg.add("DataFrames")

    Updating registry at `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/registries/General.toml`
   Resolving package versions...
   Installed Tables ────────────────────── v1.10.0
   Installed DataValueInterfaces ───────── v1.0.0
   Installed DataFrames ────────────────── v1.4.2
   Installed Crayons ───────────────────── v4.1.1
   Installed InvertedIndices ───────────── v1.2.0
   Installed TableTraits ───────────────── v1.0.1
   Installed PooledArrays ──────────────── v1.4.2
   Installed PrettyTables ──────────────── v2.2.2
   Installed IteratorInterfaceExtensions ─ v1.0.0
   Installed StringManipulation ────────── v0.3.0
    Updating `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/environments/v1.8/Project.toml`
⌃ [a93c6f00] + DataFrames v1.4.2
    Updating `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/environments/v1.8/Manifest.toml`
  [a8cc5b0e] + Crayons v4.1.1
⌃ [a93c6f00] + DataFrames v1.4.2
  [e2d170a0] + DataValueInterfaces v1.0.0
  [41ab1584] 

In [55]:
Pkg.add("CSV")

   Resolving package versions...
   Installed InlineStrings ── v1.3.2
   Installed WeakRefStrings ─ v1.4.2
   Installed SentinelArrays ─ v1.3.16
   Installed FilePathsBase ── v0.9.20
   Installed CSV ──────────── v0.10.7
    Updating `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/environments/v1.8/Project.toml`
  [336ed68f] + CSV v0.10.7
    Updating `~/Documents/master/MAOA/julia-1.8.2/.julia_commun_a_tous/environments/v1.8/Manifest.toml`
  [336ed68f] + CSV v0.10.7
  [48062228] + FilePathsBase v0.9.20
  [842dd82b] + InlineStrings v1.3.2
  [91c51154] + SentinelArrays v1.3.16
  [ea10d353] + WeakRefStrings v1.4.2
Precompiling project...
  ✓ SentinelArrays
  ✓ FilePathsBase
  ✓ InlineStrings
  ✓ WeakRefStrings
  ✓ CSV
  5 dependencies successfully precompiled in 5 seconds. 220 already precompiled. 1 skipped during auto due to previous errors.


In [69]:
include("src/readPRP.jl")
include("src/utils.jl")
include("src/Heuristic/binPacking.jl")
include("src/Heuristic/clarkWright.jl")
include("src/Heuristic/sectorielle.jl")
include("src/PLNE/PDI_BC.jl")

PDI_BC (generic function with 1 method)

In [70]:
using Dates
using CSV
using DataFrames

In [71]:
function Cout_Circuit(cout, circuit)
    c = circuit[1]
    cout_total = 0
    for (from, to) in zip(c[1:end-1], c[2:end])
        cout_total += cout[from+1, to+1]
    end
    cout_total += cout[c[end]+1, c[1]+1]
    return cout_total
end

Cout_Circuit (generic function with 1 method)

In [48]:
function test_heuristics(nb_files, dir)
    plne=false
    res = ""
    log = string("log_", Dates.now(), ".txt")
    files = readdir(dir)[1:nb_files]
    
    df = DataFrame(file = [], 
                BP_score = [],
                CW_score = [],
                SECT_score = [],
                BP_time = [],
                CW_time = [],
                SECT_time = [],
                )
    
    for file in files
        f = string(dir, "/", file)
        h = Read_PRP_instance(f)
        s = zeros(4)
        time = zeros(4)
        if plne
            pl, t_pdi = @timed PDI_BC(h)
            x = all_variables(pl)
            x = x[end-(h["n"]+1)*(h["n"]+1)*h["l"]+1:end]
        end
        
        costs = calcul_dist(h)
        for t in 1:h["l"]
            sect, t_sect = @timed Sectorielle(h, t)
            cw, t_cw = @timed clark_wright(h, h["Clients"].d, costs, t)
            bp, t_bp = @timed binPacking(h, t)
            
            if plne
                c_i = 1
                c_j = 1
                c_plne = 0
                for i in (h["n"]+1)*(h["n"]+1)*(t-1)+1 : (h["n"]+1)*(h["n"]+1)*(t)
                    c_plne += value.(x[i]) * costs[c_i, c_j]
                    c_i = (c_i + 1) % (h["n"]+2)
                    if c_i == 0
                        c_i += 1
                        c_j += 1
                    end
                end
                s[4] += c_plne
                time[4] += t_pdi
            end
            c_bp = Cout_Circuit(costs, bp)
            c_cw = Cout_Circuit(costs, cw)
            c_sect = Cout_Circuit(costs, sect)
            s[1] += c_bp
            s[2] += c_cw
            s[3] += c_sect
            time[1] += t_bp
            time[2] += t_cw
            time[3] += t_sect
        end

        small = s[1]
        if plne
            toadd = string(f, " : ", "BP : ", s[1], "/", time[1], " | CW : ", s[2], "/", time[2], " | SECT : ", s[3], "/", time[3], " | PLNE : ", s[4], "/", time[4], "\n")
        else
            toadd = string(f, " : ", "BP : ", s[1], "/", time[1], " | CW : ", s[2], "/", time[2], " | SECT : ", s[3], "/", time[3], "\n")
        end
        io = open(log, "a")
        write(io, toadd)
        close(io)
    end
    return res
end

test_heuristics (generic function with 3 methods)

In [53]:
r = test_heuristics(800, "PRP_instances")

LoadError: InterruptException:

In [72]:
function test_heuristics_csv(nb_files, dir)
    plne=false
    res = ""
    log = string("log/log_", Dates.now(), ".csv")
    files = readdir(dir)[1:nb_files]
    
    df = DataFrame(file = [], 
                BP_score = [],
                CW_score = [],
                SECT_score = [],
                BP_time = [],
                CW_time = [],
                SECT_time = [],
                )
    
    for file in files
        f = string(dir, "/", file)
        h = Read_PRP_instance(f)
        s = zeros(4)
        time = zeros(4)
        if plne
            pl, t_pdi = @timed PDI_BC(h)
            x = all_variables(pl)
            x = x[end-(h["n"]+1)*(h["n"]+1)*h["l"]+1:end]
        end
        
        costs = calcul_dist(h)
        for t in 1:h["l"]
            sect, t_sect = @timed Sectorielle(h, t)
            cw, t_cw = @timed clark_wright(h, h["Clients"].d, costs, t)
            bp, t_bp = @timed binPacking(h, t)
            
            if plne
                c_i = 1
                c_j = 1
                c_plne = 0
                for i in (h["n"]+1)*(h["n"]+1)*(t-1)+1 : (h["n"]+1)*(h["n"]+1)*(t)
                    c_plne += value.(x[i]) * costs[c_i, c_j]
                    c_i = (c_i + 1) % (h["n"]+2)
                    if c_i == 0
                        c_i += 1
                        c_j += 1
                    end
                end
                s[4] += c_plne
                time[4] += t_pdi
            end
            c_bp = Cout_Circuit(costs, bp)
            c_cw = Cout_Circuit(costs, cw)
            c_sect = Cout_Circuit(costs, sect)
            
            s[1] += c_bp
            s[2] += c_cw
            s[3] += c_sect
            time[1] += t_bp
            time[2] += t_cw
            time[3] += t_sect
        end
        
        df_current = DataFrame(file = [file], BP_score = [s[1]], CW_score = [s[2]], SECT_score = [s[3]], BP_time = [time[1]], CW_time = [time[2]], SECT_time = [time[3]])
        append!(df, df_current)
    end
    CSV.write(log, df)
    
    return df
end

test_heuristics_csv (generic function with 1 method)

In [134]:
print(r)

./PRP_instances/A_014_#ABS1_15_1.prp : BP : 21744.063854635893 | CW : 15635.991220239124 | SECT : 12422.161737710303 | PLNE2837.9883372646273


In [58]:

    "./PRP_instances/B_100_instance17.prp"

"./PRP_instances/B_100_instance17.prp"

In [124]:
string("a", 10.25, " : BL ", 547, 4)

"a10.25 : BL 5474"

In [4]:
using Dates
string(Dates.now())

"2022-12-06T13:54:57.357"

In [75]:
test_heuristics_csv(900, "PRP_instances")

""

In [36]:
 readdir("PRP_instances")

1531-element Vector{String}:
 "A_014_#ABS1_15_1.prp"
 "A_014_ABS10_15_1.prp"
 "A_014_ABS10_15_2.prp"
 "A_014_ABS10_15_3.prp"
 "A_014_ABS10_15_4.prp"
 "A_014_ABS10_15_5.prp"
 "A_014_ABS11_15_1.prp"
 "A_014_ABS11_15_2.prp"
 "A_014_ABS11_15_3.prp"
 "A_014_ABS11_15_4.prp"
 "A_014_ABS11_15_5.prp"
 "A_014_ABS12_15_1.prp"
 "A_014_ABS12_15_2.prp"
 ⋮
 "B_200_instance26.prp"
 "B_200_instance27.prp"
 "B_200_instance28.prp"
 "B_200_instance29.prp"
 "B_200_instance3.prp"
 "B_200_instance30.prp"
 "B_200_instance4.prp"
 "B_200_instance5.prp"
 "B_200_instance6.prp"
 "B_200_instance7.prp"
 "B_200_instance8.prp"
 "B_200_instance9.prp"

In [41]:
readdir("PRP_instances")[1:50]

50-element Vector{String}:
 "A_014_#ABS1_15_1.prp"
 "A_014_ABS10_15_1.prp"
 "A_014_ABS10_15_2.prp"
 "A_014_ABS10_15_3.prp"
 "A_014_ABS10_15_4.prp"
 "A_014_ABS10_15_5.prp"
 "A_014_ABS11_15_1.prp"
 "A_014_ABS11_15_2.prp"
 "A_014_ABS11_15_3.prp"
 "A_014_ABS11_15_4.prp"
 "A_014_ABS11_15_5.prp"
 "A_014_ABS12_15_1.prp"
 "A_014_ABS12_15_2.prp"
 ⋮
 "A_014_ABS17_15_3.prp"
 "A_014_ABS17_15_4.prp"
 "A_014_ABS17_15_5.prp"
 "A_014_ABS18_15_1.prp"
 "A_014_ABS18_15_2.prp"
 "A_014_ABS18_15_3.prp"
 "A_014_ABS18_15_4.prp"
 "A_014_ABS18_15_5.prp"
 "A_014_ABS19_15_1.prp"
 "A_014_ABS19_15_2.prp"
 "A_014_ABS19_15_3.prp"
 "A_014_ABS19_15_4.prp"

In [77]:
Read_PRP_instance("PRP_instances/A_014_ABS19_15_2.prp")

Dict{Any, Any} with 9 entries:
  "f"       => 3000
  "Q"       => 322
  "C"       => 10000000000
  "k"       => 2085
  "Type"    => 1
  "u"       => 30
  "l"       => 6
  "Clients" => Clients(15, [574.0, 356.0, 307.0, 143.0, 604.0, 66.0, 68.0, 996.…
  "n"       => 14

In [66]:
length(readdir("PRP_instances"))

1531

In [90]:
include("src/PLNE/LSP.jl")

"function LSP(prp, SC, optim=\"CPLEX\")\n    if optim == \"HiGHS\"\n        LP = Model(HiGHS.Optimizer)\n    elseif optim == \"CPLEX\"\n        LP = Model(CPLEX.Optimizer)\n    else\n        return -1\n    end\n\n    n = prp[\"Clients\"].nb_clients      # Nombre de clients (usine inclue)" ⋯ 1463 bytes ⋯ "prp[\"Clients\"].L[i], prp[\"Q\"], sum(prp[\"Clients\"].d[i][j] for j in t:l))\n            end\n        end\n        Mt = min(prp[\"C\"], sum(sum(prp[\"Clients\"].d[i][j] for i in 2:n) for j in t:l))\n        @constraint(LP, p_dec[t] <= Mt * y_dec[t]) # (3)\n    end\n    return LP\nend"

In [91]:
PL_LSP(Read_PRP_instance("PRP_instances/A_014_ABS19_15_2.prp"), 0)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_MIP_Tolerances_Integrality              1.0000000000000001e-15
Tried aggregator 2 times.
MIP Presolve eliminated 97 rows and 36 columns.
MIP Presolve added 2 rows and 0 columns.
Aggregator did 26 substitutions.
Reduced MIP has 71 rows, 152 columns, and 293 nonzeros.
Reduced MIP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.41 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 71 rows, 152 columns, and 293 nonzeros.
Reduced MIP has 6 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.15 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.20 ticks)

        Nodes                                         Cuts/
   Node  Left     Objecti

([0.0, 0.0, 54.0, 79.0, 79.0, 107.0], [-0.0, -0.0, 1.0, 1.0, 1.0, 1.0], 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 0:15
    Dimension 2, 0:6
And data, a 16×7 Matrix{Float64}:
 0.0    0.0   0.0                …   0.0
 0.0    0.0   0.0                    0.0
 0.0   32.0  15.99999999999999       0.0
 0.0   24.0  17.999999999999993      0.0
 0.0   20.0  14.999999999999993      0.0
 0.0   18.0   8.999999999999993  …   0.0
 0.0  105.0  83.99999999999997      -9.237055564881302e-14
 0.0   40.0  31.999999999999986     -3.730349362740526e-14
 0.0  100.0  79.99999999999997      -9.237055564881302e-14
 0.0   22.0   0.0                    0.0
 0.0   70.0  55.99999999999997   …  -7.460698725481052e-14
 0.0   19.0   0.0                    0.0
 0.0   13.0   0.0                    0.0
 0.0   68.0  50.99999999999997       0.0
 0.0   65.0  51.99999999999997      -6.394884621840902e-14
 0.0   70.0  55.99999999999997   …  -7.460698725481052e-14, [0.0 0.0 … 0.0 0.0; 0.0 0